In [49]:
import numpy as np
import pandas as pd
import ast
from sklearn.preprocessing import MultiLabelBinarizer

In [36]:
allMovies = pd.read_csv('../../Data/serendipity-sac2018/movies.csv', 
                     error_bad_lines=False)
imdbData = pd.read_csv('../Intermediate_data/IMDb_data.csv')
trainClusters = pd.read_csv('../Intermediate_data/train_clusters.csv')

Skipping line 19833: expected 8 fields, saw 10
Skipping line 34143: expected 8 fields, saw 9
Skipping line 36015: expected 8 fields, saw 10
Skipping line 37260: expected 8 fields, saw 12
Skipping line 44379: expected 8 fields, saw 10
Skipping line 47551: expected 8 fields, saw 10



In [37]:
trainClusters.head()

,Unnamed: 0,index,userId,movieId,rating,timestamp,year,title,releaseDate,directedBy,...,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,nan,cluster
0,0,25353,142748,2762,5.0,2017-12-31 23:59:41,2017,"Sixth Sense, The (1999)",1999-08-06,M. Night Shyamalan,...,0,0,1,0,0,0,0,0,0,0
1,1,26123,142752,2762,4.0,2017-12-31 20:41:29,2017,"Sixth Sense, The (1999)",1999-08-06,M. Night Shyamalan,...,0,0,1,0,0,0,0,0,0,3
2,2,26180,142744,2762,4.0,2017-12-31 20:35:25,2017,"Sixth Sense, The (1999)",1999-08-06,M. Night Shyamalan,...,0,0,1,0,0,0,0,0,0,1
3,3,26257,142753,2762,4.5,2017-12-31 20:19:35,2017,"Sixth Sense, The (1999)",1999-08-06,M. Night Shyamalan,...,0,0,1,0,0,0,0,0,0,0
4,4,26651,142747,2762,3.0,2017-12-31 18:02:45,2017,"Sixth Sense, The (1999)",1999-08-06,M. Night Shyamalan,...,0,0,1,0,0,0,0,0,0,0


In [38]:
imdbData.head()

,certificates,country codes,genres,kind,language codes,plot,rating,runtimes,votes,year,imdbId
0,"[u'Argentina:Atp', u'Australia:G', u'Belgium:K...",[u'us'],"[u'Animation', u'Adventure', u'Comedy', u'Fami...",movie,[u'en'],"[u""A cowboy doll is profoundly threatened and ...",8.3,[u'81'],837115.0,1995.0,114709
1,"[u'Argentina:Atp', u'Australia:PG', u'Brazil:1...",[u'us'],"[u'Adventure', u'Comedy', u'Family', u'Fantasy']",movie,"[u'en', u'fr']",[u'When two kids find and play a magical board...,7.0,[u'104'],287007.0,1995.0,113497
2,"[u'Argentina:Atp', u'Australia:PG', u'Finland:...",[u'us'],"[u'Comedy', u'Romance']",movie,"[u'en', u'it', u'de']","[u""John and Max resolve to save their beloved ...",6.7,[u'101'],23107.0,1995.0,113228
3,"[u'Argentina:16', u'Australia:M', u'Canada:14A...",[u'us'],"[u'Comedy', u'Drama', u'Romance']",movie,[u'en'],"[u""Based on Terry McMillan's novel, this film ...",5.9,[u'124'],9034.0,1995.0,114885
4,"[u'Argentina:Atp', u'Australia:G', u'Finland:S...",[u'us'],"[u'Comedy', u'Family', u'Romance']",movie,[u'en'],[u'George Banks must deal not only with the pr...,6.0,[u'106'],32568.0,1995.0,113041


## Format IMDb features

In [39]:
imdbData = imdbData.dropna()

In [40]:
imdbData['country codes'] = imdbData['country codes'].apply(lambda x: [str(s) for s in ast.literal_eval(x)])
imdbData['language codes'] = imdbData['language codes'].apply(lambda x: [str(s) for s in ast.literal_eval(x)])
imdbData['genres'] = imdbData['genres'].apply(lambda x: [str(s) for s in ast.literal_eval(x)])
imdbData['runtimes'] = imdbData['runtimes'].apply(lambda x: [int(s) for s in ast.literal_eval(x)][0])

In [41]:
imdbData.head()

,certificates,country codes,genres,kind,language codes,plot,rating,runtimes,votes,year,imdbId
0,"[u'Argentina:Atp', u'Australia:G', u'Belgium:K...",[us],"[Animation, Adventure, Comedy, Family, Fantasy]",movie,[en],"[u""A cowboy doll is profoundly threatened and ...",8.3,81,837115.0,1995.0,114709
1,"[u'Argentina:Atp', u'Australia:PG', u'Brazil:1...",[us],"[Adventure, Comedy, Family, Fantasy]",movie,"[en, fr]",[u'When two kids find and play a magical board...,7.0,104,287007.0,1995.0,113497
2,"[u'Argentina:Atp', u'Australia:PG', u'Finland:...",[us],"[Comedy, Romance]",movie,"[en, it, de]","[u""John and Max resolve to save their beloved ...",6.7,101,23107.0,1995.0,113228
3,"[u'Argentina:16', u'Australia:M', u'Canada:14A...",[us],"[Comedy, Drama, Romance]",movie,[en],"[u""Based on Terry McMillan's novel, this film ...",5.9,124,9034.0,1995.0,114885
4,"[u'Argentina:Atp', u'Australia:G', u'Finland:S...",[us],"[Comedy, Family, Romance]",movie,[en],[u'George Banks must deal not only with the pr...,6.0,106,32568.0,1995.0,113041


In [42]:
imdbData["kind"].value_counts()

movie             39500
tv movie           2357
video movie        1216
tv mini series      345
episode             132
tv series            85
tv short             62
Name: kind, dtype: int64

In [43]:
trainClusters.columns

Index([u'Unnamed: 0', u'index', u'userId', u'movieId', u'rating', u'timestamp',
       u'year', u'title', u'releaseDate', u'directedBy', u'starring',
       u'imdbId', u'tmdbId', u'genres', u'Action', u'Adventure', u'Animation',
       u'Children', u'Comedy', u'Crime', u'Documentary', u'Drama', u'Fantasy',
       u'Film-Noir', u'Horror', u'IMAX', u'Musical', u'Mystery', u'Romance',
       u'Sci-Fi', u'Thriller', u'War', u'Western', u'nan', u'cluster'],
      dtype='object')

In [68]:
trainData = trainClusters.drop(['genres','Unnamed: 0'],axis=1).merge(imdbData, on = 'imdbId')

In [69]:
trainData["imdbRating"] = trainData["rating_y"]
trainData["userRating"] = trainData["rating_x"]
trainData["watchYear"] = trainData["year_x"]
trainData["releaseYear"] = trainData["year_y"]
trainData.drop(['rating_x', 'rating_y', 'year_x','year_y'], axis=1,inplace=True)
trainData.head()

,index,userId,movieId,timestamp,title,releaseDate,directedBy,starring,imdbId,tmdbId,...,genres,kind,language codes,plot,runtimes,votes,imdbRating,userRating,watchYear,releaseYear
0,25353,142748,2762,2017-12-31 23:59:41,"Sixth Sense, The (1999)",1999-08-06,M. Night Shyamalan,"Haley Joel Osment, Bruce Willis, Toni Collette...",167404,745.0,...,"[Drama, Mystery, Thriller]",movie,"[en, la, es]",[u'A boy who communicates with spirits seeks t...,107,871881.0,8.1,5.0,2017,1999.0
1,26123,142752,2762,2017-12-31 20:41:29,"Sixth Sense, The (1999)",1999-08-06,M. Night Shyamalan,"Haley Joel Osment, Bruce Willis, Toni Collette...",167404,745.0,...,"[Drama, Mystery, Thriller]",movie,"[en, la, es]",[u'A boy who communicates with spirits seeks t...,107,871881.0,8.1,4.0,2017,1999.0
2,26180,142744,2762,2017-12-31 20:35:25,"Sixth Sense, The (1999)",1999-08-06,M. Night Shyamalan,"Haley Joel Osment, Bruce Willis, Toni Collette...",167404,745.0,...,"[Drama, Mystery, Thriller]",movie,"[en, la, es]",[u'A boy who communicates with spirits seeks t...,107,871881.0,8.1,4.0,2017,1999.0
3,26257,142753,2762,2017-12-31 20:19:35,"Sixth Sense, The (1999)",1999-08-06,M. Night Shyamalan,"Haley Joel Osment, Bruce Willis, Toni Collette...",167404,745.0,...,"[Drama, Mystery, Thriller]",movie,"[en, la, es]",[u'A boy who communicates with spirits seeks t...,107,871881.0,8.1,4.5,2017,1999.0
4,26651,142747,2762,2017-12-31 18:02:45,"Sixth Sense, The (1999)",1999-08-06,M. Night Shyamalan,"Haley Joel Osment, Bruce Willis, Toni Collette...",167404,745.0,...,"[Drama, Mystery, Thriller]",movie,"[en, la, es]",[u'A boy who communicates with spirits seeks t...,107,871881.0,8.1,3.0,2017,1999.0


In [70]:
trainData.columns

Index([u'index', u'userId', u'movieId', u'timestamp', u'title', u'releaseDate',
       u'directedBy', u'starring', u'imdbId', u'tmdbId', u'Action',
       u'Adventure', u'Animation', u'Children', u'Comedy', u'Crime',
       u'Documentary', u'Drama', u'Fantasy', u'Film-Noir', u'Horror', u'IMAX',
       u'Musical', u'Mystery', u'Romance', u'Sci-Fi', u'Thriller', u'War',
       u'Western', u'nan', u'cluster', u'certificates', u'country codes',
       u'genres', u'kind', u'language codes', u'plot', u'runtimes', u'votes',
       u'imdbRating', u'userRating', u'watchYear', u'releaseYear'],
      dtype='object')

In [48]:
featureCols = [
    #Movie Genres
    'Action', 'Adventure', 'Animation', 'Children',
    'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir',
    'Horror', 'IMAX', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller',
    'War', 'Western',
    #Additional Features
    'cluster', 'country codes', 'kind', 'language codes', 'runtimes', 'votes', 'imdbRating', 'releaseYear'
              ]

In [71]:
mlb = MultiLabelBinarizer()
X = mlb.fit_transform(trainData['country codes'])
X = pd.DataFrame(X, columns=mlb.classes_)
movieCodesToUse = X[mlb.classes_].sum().sort_values(ascending=False, inplace=False)
movieCodesToUse = movieCodesToUse.index[:10]
movieCodesToUse
trainData = trainData.join(X[movieCodesToUse])

In [72]:
trainData.columns

Index([u'index', u'userId', u'movieId', u'timestamp', u'title', u'releaseDate',
       u'directedBy', u'starring', u'imdbId', u'tmdbId', u'Action',
       u'Adventure', u'Animation', u'Children', u'Comedy', u'Crime',
       u'Documentary', u'Drama', u'Fantasy', u'Film-Noir', u'Horror', u'IMAX',
       u'Musical', u'Mystery', u'Romance', u'Sci-Fi', u'Thriller', u'War',
       u'Western', u'nan', u'cluster', u'certificates', u'country codes',
       u'genres', u'kind', u'language codes', u'plot', u'runtimes', u'votes',
       u'imdbRating', u'userRating', u'watchYear', u'releaseYear', u'us',
       u'gb', u'de', u'fr', u'ca', u'jp', u'nz', u'au', u'es', u'it'],
      dtype='object')

In [77]:
X = mlb.fit_transform(trainData['language codes'])
X = pd.DataFrame(X, columns=mlb.classes_)
langCodesToUse = X[mlb.classes_].sum().sort_values(ascending=False, inplace=False)
langCodesToUse = langCodesToUse.index[:10]

##Adding an l_ to the language codes to avoid clash with country codes
X.columns = ["l_" + l for l in X.columns]
langCodesToUse = ["l_" + l for l in langCodesToUse]
trainData = trainData.join(X[langCodesToUse])

In [79]:
X = mlb.fit_transform(trainData['kind'])
mlb.classes_

array([' ', 'd', 'e', 'h', 'i', 'm', 'n', 'o', 'p', 'r', 's', 't', 'v'],
      dtype=object)

In [81]:
trainData['kind'].value_counts()

movie             1177251
video movie          7070
tv movie             4487
tv mini series       3345
episode              2166
tv series            1900
tv short              457
Name: kind, dtype: int64

In [85]:
trainData['kind'] = trainData['kind'].apply(lambda x: 'movie' if x in ['movie', 'video movie'] else 'tv')

In [86]:
trainData['kind'].value_counts()

movie    1184321
tv         12355
Name: kind, dtype: int64

In [89]:
X = mlb.fit_transform(trainData['kind'].astype('category'))

In [127]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
X = enc.fit_transform(np.array(trainData['kind']).reshape(-1, 1))
X = pd.DataFrame(X.toarray(), columns=enc.categories_)
trainData = trainData.join(X[enc.categories_])

In [ ]:
def US_certificate(certificate):
    if len(certificate.split('United States')) < 2:
        return(np.nan)
    else:
        later_txt = certificate.split('United States')[1]
        UScert = re.match(r'\:(.*)',later_txt.split('\'')[0] ).group(1)
        return(UScert)